In [ ]:
import pandas as pd
import numpy as np

rxn = "borylation"
rxn = "dioxirane"
if rxn == "dioxirane":
    df     = pd.read_csv('../active_learning/regression/perf_clean_run.csv')
    max_   = 135
    rxn_lm = 50
elif rxn == "borylation":
    df     = pd.read_csv('../active_learning/regression/perf_clean_run_borylation_db0-01_a1.csv')
    max_   = 60
    rxn_lm = 22

print(f"Random reaching accuracy: {rxn_lm - df.random.to_list().count(135)}")

In [10]:
def get_avg_vs_random(af, df=df, output="mean", max_=135):
    df_ = df[[af, 'random']]
    diff = []
    for i in range(len(df_)):
        if df_[af][i] == max_ and df_['random'][i] == max_: # both are not reaching the accuracy any points
            diff.append(np.nan)
        else:
            diff.append(df_['random'][i] - df_[af][i]) # one of them is reaching the accuracy
    if output == "median":
        return np.median(diff)
    elif output == "mean":
        return np.mean(diff)
    elif output == "diff":
        return diff

In [ ]:
df_diff = pd.DataFrame()
for col in df.columns:
    if 'acqf' in col:
        df_diff[col] = get_avg_vs_random(col, df, output="diff", max_=max_)
df_diff["SMILES"] = df["Unnamed: 0"]

In [ ]:
df_results_acqf = pd.DataFrame()
non_zero_counts = rxn_lm - df_diff.isna().sum()
df_results_acqf["AF"] = df_diff.columns
df_results_acqf['num_target_acc']            = non_zero_counts.values
df_results_acqf['mean_vs_rd']                = list(df_diff.mean())
df_results_acqf['median_vs_rd']              = list(df_diff.median())

df_results_acqf
#df_results_acqf.to_csv(f"acqf_results_{rxn}.csv")

In [13]:
df = df[df['random'] != max_]
#df.drop(columns=['Unnamed: 0'], inplace=True)
df_copy = df.copy() 
for col in df_copy.columns:
    if 'acqf' in col:
        df_copy[col] = df[col] - df['random']


In [ ]:
random_data = df.random.mean()
for col in df_copy.columns:
    if 'acqf' in col:
        print(col, round(100*df_copy[col].mean()/random_data,1), "% of random")